In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

### import data

def txt_to_df(filename):
    
    path = "/home/hermuba/res/data/data_from_dr_wu/"
    i = 0
    with open(path + filename) as f:
        for line in f:
            l = line.replace('\n', '').split('\t')
            if i == 0:
                l[0] = "Genome ID"
                l[1] = "Resistant Phenotype"
                df = pd.DataFrame(columns = l)
 
            else:
                ID = l[0]
                
                df.loc[i-1, :] = l
            i += 1
    return(df)

/home/hermuba/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:279: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [3]:
#those drugs have nearly 50 data, therefore their data are selected for training
train_drug = ['meropenem','cefepime','ceftazidime' 'gentamicin', 'ciprofloxacin',
              'trimethoprim_sulfamethoxaole', 'ampicillin', 'cefazolin', 'ampicillin_sulbactam']
prefix = ['card_pattern', 'gene_pattern', 'acc_pattern', 'acc_card_pattern']

In [21]:
def random_split(df, portion):
    arr = np.arange(len(df.index))
    np.random.shuffle(arr)
    shuffle_index = arr
    train_size = round(len(df.index)*portion)
    
    return(shuffle_index[:train_size], shuffle_index[train_size:])


In [14]:
# throw into naive bayes
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score     
def train_nb(df_ris, portion):
    # hold out test set
    train_index, test_index = random_split(df_ris, portion)
    X = df_ris.drop(["Genome ID", "Resistant Phenotype"], axis = 1)
    y = df_ris['Resistant Phenotype']
    X_train = X.loc[train_index, :]
    X_test = X.loc[test_index, :]
    y_train = y[train_index]
    y_test = y[test_index]
        
        #X_train, X_test, y_train, y_test = train_test_split(
        #df_ris.drop(["Genome ID", "Resistant Phenotype"], axis = 1)
        #, df_ris['Resistant Phenotype']
        #, test_size=0.2 
        #, random_state = 1
        #, startify = df_ris['Resistant Phenotype']) 
    # test if random
    print(y_test)
    ### i dont really know pseudo random status mean (please read about it), random status 20 yields best result?? 0.7
    ### it saids it is "random sampling test set but it is not (WTF)
    
    X_train.reset_index(inplace = True, drop = True)
    y_train.reset_index(inplace = True, drop = True)
    
    # seperate train and test into k fold
    n=10
    cv = ShuffleSplit(n_splits=n, test_size=0.3, random_state = 0)
    ### n_split = 10, test_size, does not change result; random_state yeilds best result 0.75
    
    
    # bnb.fit
    bnb = BernoulliNB()
    val_score = 0
    for train_index, test_index in cv.split(X_train, y_train):      
        bnb.fit(X_train.iloc[train_index,:], y_train[train_index])
        val = bnb.predict(X_train.iloc[test_index])
        score = accuracy_score(y_train[test_index], val)
        val_score = val_score + score
    
    v = val_score/n
    
    # model.predict()
    
    test = accuracy_score(y_test, bnb.predict(X_test))
    
    return([v,test])
   

In [24]:
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold # this is the problem!!

    
def train_SVM(df, portion):
    
    # split test, train
    train_index, test_index = random_split(df_ris, p)
    X = df_ris.drop(["Genome ID", "Resistant Phenotype"], axis = 1)
    y = df_ris['Resistant Phenotype']
    X_train = X.loc[train_index, :]
    X_test = X.loc[test_index, :]
    y_train = y[train_index]
    y_test = y[test_index]
    
    # choose estimator (our model)
    clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
    
    # cross validation
    skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
    cv = skf.split(X_train, y_train)
   
    # tune hyperparameters
    gammas = np.logspace(-6, -1, 10)
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': gammas,
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    classifier = GridSearchCV(estimator=clf, cv=cv, param_grid=tuned_parameters)
    classifier.fit(X_train.values, np.asarray(y_train))
    
    v = classifier.cv_results_['mean_test_score'][classifier.best_index_]
    param = classifier.best_params_
    print(param)
    test = classifier.score(X_test, y_test)
    
    return([v,test])

In [23]:
train_nb(txt_to_df("gene_pattern.trimethoprim_sulfamethoxazole"))

3       Resistant
5       Resistant
11    Susceptible
24    Susceptible
25      Resistant
0     Susceptible
19      Resistant
28      Resistant
Name: Resistant Phenotype, dtype: object


[0.70909090909090911, 0.625]

In [18]:
print(np.random.shuffle([1,2,3,4,5]))

None
